In [2]:
import numpy as np
import tensorflow as tf

import cv2

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


from keras.callbacks import LearningRateScheduler
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout, Activation, Conv2D, MaxPool2D

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train/255.0
x_test = x_test/255.0


x_train = x_train.tolist()
y_train = y_train.tolist()

x_1 = cv2.imread("img-07.png", 0) / 255.0
x_6 = cv2.imread("img-85.png", 0) / 255.0

for i in range(700):
    x_train.append(np.zeros((28, 28), dtype=np.float64))
    y_train.append(0)
    x_train.append(np.array(x_1))
    y_train.append(1)
    x_train.append(np.array(x_6))
    y_train.append(6)

    
x_train = np.array(x_train)
y_train = np.array(y_train)

x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.333)

epochs = 20
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** (x+epochs))

In [5]:
model = Sequential()

model.add(Conv2D(32,kernel_size=3,activation='relu',input_shape=(28,28,1)))
model.add(BatchNormalization())
model.add(Conv2D(32,kernel_size=3,activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(32,kernel_size=5,strides=2,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(64,kernel_size=3,activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64,kernel_size=3,activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64,kernel_size=5,strides=2,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 26, 26, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 32)        25632     
_________________________________________________________________
batch_normalization_2 (Batch (None, 12, 12, 32)        128       
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 32)        0

In [6]:
datagen = ImageDataGenerator(
        rotation_range=5,  
        zoom_range = 0.1,
        width_shift_range=0.1,
        height_shift_range=0.1)

epochs = 20

history = model.fit(datagen.flow(x_train, y_train, batch_size=64), 
    epochs = epochs, steps_per_epoch = x_train.shape[0]//64,
    validation_data = (x_val, y_val), callbacks=[annealer], verbose=1, shuffle=True)

Epoch 1/20
647/647 [==============================] - 18s 24ms/step - loss: 0.8163 - accuracy: 0.7440 - val_loss: 0.0957 - val_accuracy: 0.9703
Epoch 2/20
647/647 [==============================] - 15s 23ms/step - loss: 0.2125 - accuracy: 0.9351 - val_loss: 0.0686 - val_accuracy: 0.9794
Epoch 3/20
647/647 [==============================] - 15s 24ms/step - loss: 0.1408 - accuracy: 0.9580 - val_loss: 0.0788 - val_accuracy: 0.9696
Epoch 4/20
647/647 [==============================] - 15s 24ms/step - loss: 0.1089 - accuracy: 0.9677 - val_loss: 0.0660 - val_accuracy: 0.9808
Epoch 5/20
647/647 [==============================] - 15s 24ms/step - loss: 0.0905 - accuracy: 0.9726 - val_loss: 0.0352 - val_accuracy: 0.9899
Epoch 6/20
647/647 [==============================] - 15s 23ms/step - loss: 0.0733 - accuracy: 0.9778 - val_loss: 0.0427 - val_accuracy: 0.9870
Epoch 7/20
647/647 [==============================] - 16s 24ms/step - loss: 0.0699 - accuracy: 0.9794 - val_loss: 0.0339 - val_accuracy:

In [7]:
model.evaluate(x_test, y_test)

y_pred = model.predict(x_test)

y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)

print(y_pred.shape, y_test.shape)

print(classification_report(y_test, y_pred))

313/313 [==============================] - 1s 5ms/step - loss: 0.0145 - accuracy: 0.9953
(10000,) (10000,)
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       980
           1       0.99      1.00      1.00      1135
           2       1.00      1.00      1.00      1032
           3       0.99      1.00      0.99      1010
           4       0.99      0.99      0.99       982
           5       1.00      0.99      0.99       892
           6       1.00      0.99      1.00       958
           7       1.00      0.99      0.99      1028
           8       1.00      1.00      1.00       974
           9       0.99      0.99      0.99      1009

    accuracy                           1.00     10000
   macro avg       1.00      1.00      1.00     10000
weighted avg       1.00      1.00      1.00     10000



In [8]:
model.save('../Keras-Models/CNN+BN+DA-5')

INFO:tensorflow:Assets written to: ../Keras-Models/CNN+BN+DA-5/assets


In [1]:
from keras.models import load_model
import cv2
import random

model = load_model("digit_model.h5")


In [4]:

i = random.randint(0, 500)
x = np.zeros((28, 28))+255
cv2.imshow("img", x)
cv2.imshow("test", x_test[i].reshape(28, 28))


x = x.reshape(-1, 28, 28, 1)
x = tf.keras.utils.normalize(x, axis=1)
# print(x.shape)
# x = x.reshape(-1, 28, 28, 1)
pred = model.predict(x)
print(pred)

# model.evaluate(x_test, y_test)

y_pred = model.predict(x_test[i].reshape(-1, 28, 28, 1))
print(y_pred)
# y_pred = np.argmax(y_pred, axis=1)

# print(classification_report(y_test, y_pred))

cv2.waitKey(0)
cv2.destroyAllWindows()

[[0.45674202 0.5944998  0.4800058  0.45394    0.5175149  0.4436968
  0.4751583  0.51630557 0.5160351  0.42816165]]
[[7.4413083e-06 2.1672030e-03 5.7088761e-07 9.8061025e-01 2.2372857e-05
  1.0000000e+00 4.5162495e-03 1.3231435e-04 9.3374252e-03 1.8740301e-01]]


In [7]:
data = np.load("data.npy")
data.shape

(81, 28, 28, 1)

In [9]:
cv2.imshow("test", data[3])

# model.predict()

cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
img = cv2.imread("img-26.png", 0)
x = img.reshape(-1, 28, 28, 1)
x = x/255.0